pip install pgeocode

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
import pgeocode
from sklearn.metrics import DistanceMetric
from math import radians
import pgeocode
import warnings
warnings.filterwarnings('ignore')    

In [3]:
def process_input():
    input_df = pd.read_excel(r"C:\Users\User\Desktop\GREENPLY\GreenPlyInput_Logiteg 1.xlsx", sheet_name= None)
    gs04_input_data = input_df['trips'].copy(deep=True)
    gs03_input_data = input_df['GS03'].copy(deep=True)
    gs02_input_data = input_df['GS02'].copy(deep=True)
    gs04_input_data['Indent Date'] = pd.to_datetime(gs04_input_data['Indent Date'], format='%d.%m.%Y')
    gs04_input_data['Indent Date'] = gs04_input_data['Indent Date'].dt.date
    gs04_input_data['todayDate'] = pd.to_datetime(gs04_input_data['todayDate'], format='%d.%m.%Y')
    gs04_input_data['Expected DispatchD'] = gs04_input_data['todayDate'].dt.date
    df2 = gs04_input_data[gs04_input_data["rdd"]=="0000-00-00"]

    df2
    gs04_input_data['rdd'] = pd.to_datetime(gs04_input_data['rdd'], errors='coerce')
    gs04_input_data['current_date'] = pd.Timestamp.now()
    gs04_input_data = gs04_input_data[gs04_input_data['current_date'] >= gs04_input_data['rdd']]
    gs04_input_data = pd.concat([gs04_input_data, df2], ignore_index=True)
    gs04_input_data['rdd'].unique()
    gs04_input_data 
    return gs04_input_data, gs03_input_data, gs02_input_data
gs04_input_data,gs03_input_data,gs02_input_data= process_input()
# gs04_input_data

In [4]:
def cleaning_data(gs04_input_data):
    gs04_input_data['Plant'] = gs04_input_data['Supply Plant'] 
    gs04_input_data['Pin code '] = gs04_input_data['Ship-to-party Pin Code']
    gs04_input_data['Order Qty'] = gs04_input_data['Pending Qty']
    gs04_input_data['Stock Quantity'] = gs04_input_data['Stock']
    gs04_input_data['Gross Weight'] = gs04_input_data['Pending Weight']
    gs04_input_data['Order CBM'] = gs04_input_data['Pending CBM']  
    gs04_input_data['Ship-To-Party City'] = gs04_input_data['Ship-To-Party City'].str.upper()
    gs04_input_data['Ship-To-Party State'] = gs04_input_data['shipToPartyState'].str.upper()
    gs04_input_data = gs04_input_data.drop(columns=[
        'Supply Plant', 
        'Ship-to-party Pin Code', 
        'Pending Qty', 
        'Stock', 
        'Pending Weight', 
        'Pending CBM', 
        'shipToPartyState'
    ])

    return gs04_input_data 
data=cleaning_data(gs04_input_data)
# data

In [5]:
def process_vehicle_master():
    input_df = pd.read_excel(r"C:\Users\User\Desktop\GREENPLY\GreenPlyVehicleMaster 1.xlsx", sheet_name= None)
    thin_material_full = input_df['Thin_full'].copy(deep=True) #df2
    thin_material_full = thin_material_full.replace(1,np.inf)   
    thin_full_size_10x4 = input_df['Thin_full_10x4'].copy(deep=True) #df21
    thin_full_size_10x4 = thin_full_size_10x4.replace(1,np.inf) 
    thick_material_full = input_df['Thick_full'].copy(deep=True) #df5
    thick_material_full = thick_material_full.replace(1,np.inf) 
    thick_full_size_10x4 = input_df['Thick_full_10x4'].copy(deep=True) #df19
    thick_full_size_10x4 = thick_full_size_10x4.replace(1,np.inf) 
    mixed_material = input_df['mixed_material_vehicle'].copy(deep=True) #df9
    mixed_material = mixed_material.replace(1,np.inf) 
    mixed_material_10x4 = input_df['mixed_material_vehicle_10x4'].copy(deep=True) #df91
    mixed_material_10x4 = mixed_material_10x4.replace(1,np.inf) 
    north_east_thick_material_fulll = input_df['north_east_zone_thick_full_23MT'].copy(deep=True) #df55
    north_east_thick_material_fulll = north_east_thick_material_fulll.replace(1,np.inf) 
    south_thick_material_full = input_df['southzone24MT_30MT_thick'].copy(deep=True) #df44
    south_thick_material_full = south_thick_material_full.replace(1,np.inf)
    south_thin_material_full = input_df['southzone24MT_thin'].copy(deep=True) #df66
    south_thin_material_full = south_thin_material_full.replace(1,np.inf)
    kerala_tn_material = input_df['keralTamilnadu23Mt'].copy(deep=True) # df105
    kerala_tn_material = kerala_tn_material.replace(1,np.inf)
    east_mixed_material = input_df['East_zone_thin_thick'].copy(deep=True) # df77
    east_mixed_material = east_mixed_material.replace(1,np.inf)
    route_master = input_df['updated_route_master'].copy(deep=True) # df7
    return thin_material_full,thin_full_size_10x4,thick_material_full,thick_full_size_10x4,mixed_material,mixed_material_10x4,north_east_thick_material_fulll,south_thick_material_full,south_thin_material_full,kerala_tn_material,east_mixed_material,route_master    
    
thin_material_full,thin_full_size_10x4,thick_material_full,thick_full_size_10x4,mixed_material,mixed_material_10x4,north_east_thick_material_fulll,south_thick_material_full,south_thin_material_full,kerala_tn_material,east_mixed_material, route_master = process_vehicle_master()                   

In [6]:
def material_bifurcation(data):
    data['Thickness'] = data['Thickness'].astype(str).str.extract('(\d+\.\d+)').astype(float)
    data['Thin'] = 0
    data.loc[data['Thickness'] <= 6, 'Thin'] = 1 
    data['Type'] = np.where(data['Thin'] == 1, 'Thin', 'Thick')  
    return data
    
data=material_bifurcation(data)
# data

In [7]:
def mapping_tat_distance(route_master, data):
    lookup_dict = route_master.set_index('Ship-To-Party City').to_dict()
    data['TAT'] = data['Ship-To-Party City'].map(lookup_dict['TAT'])
    data['Distance'] = data['Ship-To-Party City'].map(lookup_dict['Distance'])
    data.sort_values(by=['Material','Indent No.'], inplace=True)
    data.reset_index(drop=True, inplace=True) 
    return data

data = mapping_tat_distance(route_master, data)

In [8]:
def gs04_remaining_stock(data):
    def calculate_available_stock(group):
        group['available_stock'] = group['Stock Quantity'].iloc[0] - group['Order Qty'].cumsum()
        return group
    data = data.groupby(['Material']).apply(calculate_available_stock).reset_index(drop=True)
    data['Gs04_remaining_stock'] = data['available_stock'] 
    data.loc[data['Gs04_remaining_stock'] < 0, 'Gs04_remaining_stock'] = None 
    data['Gs04_remaining_stock'] = data.groupby('Material')['Gs04_remaining_stock'].transform(lambda x: x.fillna(x.min()))
    data['Gs04_remaining_stock'].fillna(data['Stock Quantity'], inplace=True)
    available_stock_less_than_0_gs04 = data[data['available_stock'] < 0].reset_index(drop=True) #df88 = available_stock_less_than_0_gs04
    data = data[data['available_stock'] >= 0].reset_index(drop=True)
    return data,available_stock_less_than_0_gs04
    
data,available_stock_less_than_0_gs04 = gs04_remaining_stock(data)
# available_stock_less_than_0_gs04

In [9]:
def process_gs03_and_update_stock(gs03_input_data, available_stock_less_than_0_gs04):
    gs03_input_data['Grade_A'] = gs03_input_data['Stock'] * 0.95
    gs03_input_data['Grade_L'] = gs03_input_data['Stock'] * 0.02
    gs03_input_data['Grade_B'] = gs03_input_data['Stock'] * 0.01
    gs03_input_data[['Grade_A', 'Grade_L', 'Grade_B']] = gs03_input_data[['Grade_A', 'Grade_L', 'Grade_B']].astype(int)
    ABC = gs03_input_data['Grade_A'] + gs03_input_data['Grade_L'] + gs03_input_data['Grade_B']
    gs03_input_data['Grade_C'] = gs03_input_data['Stock'] - ABC
    lookup_dict = gs03_input_data.set_index('Material_Code')['Grade_A'].to_dict()
    available_stock_less_than_0_gs04['gs03_Agrade'] = available_stock_less_than_0_gs04['Material'].map(lookup_dict)
    available_stock_less_than_0_gs04['GS03_STOCK'] = available_stock_less_than_0_gs04['gs03_Agrade'] + available_stock_less_than_0_gs04['Gs04_remaining_stock']
    available_stock_less_than_0_gs04.sort_values(by=['Material', 'Indent No.'], inplace=True)
    available_stock_less_than_0_gs04.reset_index(drop=True, inplace=True)
    return available_stock_less_than_0_gs04
    
available_stock_less_than_0_gs04 = process_gs03_and_update_stock(gs03_input_data, available_stock_less_than_0_gs04)
# available_stock_less_than_0_gs04

In [10]:
def gs03_remaining_stock(available_stock_less_than_0_gs04):
    def calculate_available_stock1(group):
        group['gs03_available_stock'] = group['GS03_STOCK'].iloc[0] - group['Order Qty'].cumsum()
        return group
    available_stock_less_than_0_gs04 = available_stock_less_than_0_gs04.groupby(['Material']).apply(calculate_available_stock1).reset_index(drop=True)
    available_stock_less_than_0_gs04['Gs03_remaining_stock'] = available_stock_less_than_0_gs04['gs03_available_stock'] 
    available_stock_less_than_0_gs04.loc[available_stock_less_than_0_gs04['Gs03_remaining_stock'] < 0, 'Gs03_remaining_stock'] = None 
    available_stock_less_than_0_gs04['Gs03_remaining_stock'] = available_stock_less_than_0_gs04.groupby('Material')['Gs03_remaining_stock'].transform(lambda x: x.fillna(x.min()))
    available_stock_less_than_0_gs04['gs03_available_stock'].fillna(available_stock_less_than_0_gs04['available_stock'], inplace=True)
    available_stock_less_than_0_gs04['Gs03_remaining_stock'].fillna(available_stock_less_than_0_gs04['Gs04_remaining_stock'], inplace=True) 
    available_stock_less_than_0_gs03 = available_stock_less_than_0_gs04[available_stock_less_than_0_gs04['gs03_available_stock'] < 0].reset_index(drop=True) #df99 = available_stock_less_than_0_gs03
    maintained_stock_gs04 = available_stock_less_than_0_gs04[available_stock_less_than_0_gs04['gs03_available_stock'] >= 0].reset_index(drop=True) #df100 = maintained_stock_gs04
    return available_stock_less_than_0_gs03, maintained_stock_gs04

available_stock_less_than_0_gs03, maintained_stock_gs04 = gs03_remaining_stock(available_stock_less_than_0_gs04)
# maintained_stock_gs04

In [11]:
# maintained_stock_gs04

In [12]:
def process_gs02_and_update_stock(gs02_input_data, available_stock_less_than_0_gs03):
    gs02_input_data['Grade_A'] = gs02_input_data['stock'] * 95.0 / 100
    gs02_input_data['Grade_A'] = gs02_input_data['Grade_A'].astype(int)
    gs02_input_data['Grade_C'] = gs02_input_data['stock'] - gs02_input_data['Grade_A']    
    lookup_dict = gs02_input_data.set_index('Material_Code')['Grade_A'].to_dict()
    available_stock_less_than_0_gs03['gs02_Agrade'] = available_stock_less_than_0_gs03['Material'].map(lookup_dict) 
    available_stock_less_than_0_gs03['GS02_STOCK'] = available_stock_less_than_0_gs03['gs02_Agrade'] + available_stock_less_than_0_gs03['Gs03_remaining_stock'] 
    available_stock_less_than_0_gs03.sort_values(by=['Material','Indent No.'], inplace=True)
    available_stock_less_than_0_gs03.reset_index(drop=True, inplace=True)
    return available_stock_less_than_0_gs03

available_stock_less_than_0_gs03 = process_gs02_and_update_stock(gs02_input_data, available_stock_less_than_0_gs03)
# available_stock_less_than_0_gs03

In [13]:
def calculate_available_stock1(group):
        group['gs02_available_stock'] = group['GS02_STOCK'].iloc[0] - group['Order Qty'].cumsum()
        return group  
available_stock_less_than_0_gs03 = available_stock_less_than_0_gs03.groupby(['Material']).apply(calculate_available_stock1).reset_index(drop=True)
available_stock_less_than_0_gs03['gs02_available_stock'].fillna(available_stock_less_than_0_gs03['gs03_available_stock'], inplace=True)
data = pd.concat([maintained_stock_gs04, data, available_stock_less_than_0_gs03], ignore_index=True)
data['gs03_available_stock'].fillna(data['available_stock'], inplace=True)
data['gs02_available_stock'].fillna(data['gs03_available_stock'], inplace=True)
data['Gross Weight'] = data['Gross Weight'].mask((data['gs02_available_stock'] < 0) | (data['gs02_available_stock'].isnull()), 0)
# data

In [14]:
def calculate_sum_gw_oc(data):
    df_filtered3 = data.groupby('Indent No.').filter(lambda group: (1 in group['Thin'].values) and (0 in group['Thin'].values))
    df_fil = data[~data.index.isin(df_filtered3.index)]
    df_filtered = df_fil[df_fil['Thin'] == 1]
    df_filtered2 = df_fil[df_fil['Thin'] == 0]
   
    Sum_thin_GW = df_filtered.groupby(['Indent No.','Indent Date','todayDate','Ship-To-Party','Ship-To-Party Name','Ship-To-Party City','Ship-To-Party State','Zone','Pin code ','Sales Office','Distance','TAT']).agg({'Gross Weight': 'sum', 'Order CBM': 'sum', 'Expected DispatchD': 'max'}).reset_index()
    Sum_thick_GW = df_filtered2.groupby(['Indent No.','Indent Date','todayDate','Ship-To-Party','Ship-To-Party Name','Ship-To-Party City','Ship-To-Party State','Zone','Pin code ','Sales Office','Distance','TAT']).agg({'Gross Weight': 'sum', 'Order CBM': 'sum', 'Expected DispatchD': 'max'}).reset_index()
    Sum_mixed_GW = df_filtered3.groupby(['Indent No.','Indent Date','todayDate','Ship-To-Party','Ship-To-Party Name','Ship-To-Party City','Ship-To-Party State','Zone','Pin code ','Sales Office','Distance','TAT']).agg({'Gross Weight': 'sum', 'Order CBM': 'sum', 'Expected DispatchD': 'max'}).reset_index()
    
    return Sum_thin_GW, Sum_thick_GW, Sum_mixed_GW,df_filtered3,df_filtered,df_filtered2
Sum_thin_GW,Sum_thick_GW,Sum_mixed_GW,df_filtered3,df_filtered,df_filtered2 = calculate_sum_gw_oc(data)

In [15]:
Sum_mixed_GW

,Indent No.,Indent Date,todayDate,Ship-To-Party,Ship-To-Party Name,Ship-To-Party City,Ship-To-Party State,Zone,Pin code,Sales Office,Distance,TAT,Gross Weight,Order CBM,Expected DispatchD
0,1530012901,2024-10-07,2024-11-06,1000029557,RAVI AGENCY,HYDERABAD,TELANGANA,SOUTH,500001,SHYD,1170.0,5.0,1054.40,4.495,2024-11-06
1,1530012921,2024-10-08,2024-11-06,1000001172,TODI PLYWOOD,KOLKATA,WEST BENGAL,EAST,700012,EKOL,1960.0,8.0,3643.60,32.567,2024-11-06
2,1530012953,2024-10-08,2024-11-06,1000028745,ZEENATH PLYWOODS & CORPORATE INTERIOR,MANGALORE,KARNATAKA,SOUTH,575001,SBLR,1388.0,6.0,278.34,1.607,2024-11-06
3,1530012955,2024-10-08,2024-11-06,1000028745,ZEENATH PLYWOODS & CORPORATE INTERIOR,MANGALORE,KARNATAKA,SOUTH,575001,SBLR,1388.0,6.0,1194.62,1.438,2024-11-06
4,1530013016,2024-10-09,2024-11-06,1000028767,SHIVANGI INDUSTRIES,JAIPUR,RAJASTHAN,NORTH,303007,NJPR,626.0,3.0,0.00,24.029,2024-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,1530013717,2024-11-02,2024-11-06,1000029060,A B M WOOD DECOR PRIVATE LIMITED,SURAT,GUJARAT,WEST,394510,WSRT,246.0,2.0,0.00,23.360,2024-11-06
99,1530013720,2024-11-04,2024-11-06,1000029307,MAHESH PLYWOOD,AJMER,RAJASTHAN,NORTH,305001,NJPR,514.0,3.0,9473.92,12.782,2024-11-06
100,1530013722,2024-11-04,2024-11-06,1000002022,VISHWANATH GLASS PLY & HARDWARE,VARANASI,UTTAR PRADESH,CENTRA,221005,NLKW,1295.0,5.0,12549.43,14.690,2024-11-06
101,1530013726,2024-11-04,2024-11-06,1000029278,GLO PANELS PRIVATE LIMITED,MUMBAI,MAHARASHTRA,WEST,421601,WMUM,507.0,3.0,7514.91,11.952,2024-11-06


In [16]:
def clubbing_thin_material(Sum_thin_GW):
    Sum_thin_GW['Cluster_No'] = -1 
    Sum_thin_GW['Loading_order'] = 0 
    Sum_thin_GW['Min_Weight_Kgs'] = 0 
    Sum_thin_GW['Max_Weight_Kgs'] = 0 
    
    # Use Indent No. as Cluster_No
    Sum_thin_GW['Cluster_No'] = Sum_thin_GW['Indent No.']
    new_cluster_counter = Sum_thin_GW['Cluster_No'].max() + 1
    for city, group in Sum_thin_GW.groupby('Ship-To-Party City'):
        party_names = group['Ship-To-Party Name'].unique()
        # Create clusters with a maximum of 2 Ship-To-Party Names
        for i in range(0, len(party_names), 2):
            names_subset = party_names[i:i+2]
            subset_group = group[group['Ship-To-Party Name'].isin(names_subset)]
            Sum_thin_GW.loc[subset_group.index, 'Cluster_No'] = new_cluster_counter
            new_cluster_counter += 1 
    
    # Step 3: Further cluster single-line item clusters
    single_line_clusters = Sum_thin_GW[Sum_thin_GW.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    Sum_thin_GW = Sum_thin_GW[~Sum_thin_GW['Cluster_No'].isin(single_line_clusters['Cluster_No'])]
    
    re_clustered_df = pd.DataFrame()
    for city, group in single_line_clusters.groupby('Ship-To-Party City'):
        group = group.sort_values(by='Indent No.').reset_index(drop=True)
        clusters = np.array_split(group, np.ceil(len(group) / 2))
        for cl in clusters:
            if len(cl) > 0:
                cl['Cluster_No'] = new_cluster_counter
                re_clustered_df = pd.concat([re_clustered_df, cl], ignore_index=True)
                new_cluster_counter += 1
    Sum_thin_GW = pd.concat([Sum_thin_GW, re_clustered_df], ignore_index=True)
    
    # Step 4: Re-cluster single-line item clusters within 100 km
    single_line_clusters = Sum_thin_GW[Sum_thin_GW.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    Sum_thin_GW = Sum_thin_GW[~Sum_thin_GW['Cluster_No'].isin(single_line_clusters['Cluster_No'])]

    # Helper function to calculate distance matrix
    def get_distance_matrix(postal_codes):
        india = pgeocode.GeoDistance('IN')
        dist_mat = pd.DataFrame(index=postal_codes, columns=postal_codes)
        for i in range(len(postal_codes)):
            for j in range(i, len(postal_codes)):
                if i == j:
                    dist_mat.iloc[i, j] = 0
                else:
                    dist = india.query_postal_code(postal_codes[i], postal_codes[j])
                    dist_mat.iloc[i, j] = dist
                    dist_mat.iloc[j, i] = dist
        return dist_mat
    def cluster_within_distance(df, dist_matrix, max_distance, new_cluster_counter):
        indices_used = set()
        re_clustered_df = pd.DataFrame()

        for i in range(len(dist_matrix)):
            if i in indices_used:
                continue
            indices_used.add(i)
            cluster_indices = [i]

            for j in range(i + 1, len(dist_matrix)):
                if j in indices_used:
                    continue
                if dist_matrix.iloc[i, j] <= max_distance and len(cluster_indices) < 2:
                    cluster_indices.append(j)
                    indices_used.add(j)

            cluster_df = df.iloc[cluster_indices].copy()
            cluster_df['Cluster_No'] = new_cluster_counter
            re_clustered_df = pd.concat([re_clustered_df, cluster_df], ignore_index=True)
            new_cluster_counter += 1

        return re_clustered_df, new_cluster_counter

    # Perform re-clustering if single-line clusters are present
    if len(single_line_clusters) > 0:
        pcodes = list(single_line_clusters['Pin code '])
        dist_matrix = get_distance_matrix(pcodes)

        # Perform the re-clustering
        re_clustered_df, new_cluster_counter = cluster_within_distance(single_line_clusters, dist_matrix, 100, new_cluster_counter)
        Sum_thin_GW = pd.concat([Sum_thin_GW, re_clustered_df], ignore_index=True)

    return Sum_thin_GW
Sum_thin_GW = clubbing_thin_material(Sum_thin_GW)
# Sum_thin_GW

In [17]:
def reassign_cluster_numbers(Sum_thin_GW):
    unique_clusters = Sum_thin_GW['Cluster_No'].unique()
    if len(unique_clusters) > 100:
        raise ValueError("Number of unique clusters exceeds 100")
    cluster_mapping = {old_cluster: new_cluster for old_cluster, new_cluster in zip(unique_clusters, range(1, len(unique_clusters) + 1))}
    Sum_thin_GW['Cluster_No'] = Sum_thin_GW['Cluster_No'].map(cluster_mapping)
    return Sum_thin_GW
Sum_thin_GW = reassign_cluster_numbers(Sum_thin_GW)

In [18]:
def clubbing_thik_material(Sum_thick_GW):
    Sum_thick_GW['Cluster_No'] = -1
    Sum_thick_GW['Loading_order'] = 0
    Sum_thick_GW['Min_Weight_Kgs'] = 0
    Sum_thick_GW['Max_Weight_Kgs'] = 0
    Sum_thick_GW['Cluster_No'] = Sum_thick_GW['Indent No.']

    new_cluster_counter = Sum_thick_GW['Cluster_No'].max() + 1
    for city, group in Sum_thick_GW.groupby('Ship-To-Party City'):
        party_names = group['Ship-To-Party Name'].unique()
        for i in range(0, len(party_names), 2):
            names_subset = party_names[i:i+2]
            subset_group = group[group['Ship-To-Party Name'].isin(names_subset)]
            Sum_thick_GW.loc[subset_group.index, 'Cluster_No'] = new_cluster_counter
            new_cluster_counter += 1

    single_line_clusters = Sum_thick_GW[Sum_thick_GW.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    Sum_thick_GW = Sum_thick_GW[~Sum_thick_GW['Cluster_No'].isin(single_line_clusters['Cluster_No'])]

    new_cluster_counter = Sum_thick_GW['Cluster_No'].max() + 1
    re_clustered_df = pd.DataFrame()
    for city, group in single_line_clusters.groupby('Ship-To-Party City'):
        group = group.sort_values(by='Indent No.').reset_index(drop=True)
        clusters = np.array_split(group, np.ceil(len(group) / 2))
        for cl in clusters:
            if len(cl) > 0:
                cl['Cluster_No'] = new_cluster_counter
                re_clustered_df = pd.concat([re_clustered_df, cl], ignore_index=True)
                new_cluster_counter += 1

    Sum_thick_GW = pd.concat([Sum_thick_GW, re_clustered_df], ignore_index=True)

    single_line_clusters = Sum_thick_GW[Sum_thick_GW.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    Sum_thick_GW = Sum_thick_GW[~Sum_thick_GW['Cluster_No'].isin(single_line_clusters['Cluster_No'])]

    def get_distance_matrix(postal_codes):
        india = pgeocode.GeoDistance('IN')
        dist_mat = pd.DataFrame(index=postal_codes, columns=postal_codes)
        for i in range(len(postal_codes)):
            for j in range(i, len(postal_codes)):
                if i == j:
                    dist_mat.iloc[i, j] = 0
                else:
                    dist = india.query_postal_code(postal_codes[i], postal_codes[j])
                    dist_mat.iloc[i, j] = dist
                    dist_mat.iloc[j, i] = dist
        return dist_mat

    def cluster_within_distance(df, dist_matrix, max_distance, new_cluster_counter):
        indices_used = set()
        re_clustered_df = pd.DataFrame()

        for i in range(len(dist_matrix)):
            if i in indices_used:
                continue
            indices_used.add(i)
            cluster_indices = [i]
            
            for j in range(i + 1, len(dist_matrix)):
                if j in indices_used:
                    continue
                if dist_matrix.iloc[i, j] <= max_distance and len(cluster_indices) < 2:
                    cluster_indices.append(j)
                    indices_used.add(j)
                    
            cluster_df = df.iloc[cluster_indices].copy()
            cluster_df['Cluster_No'] = new_cluster_counter
            re_clustered_df = pd.concat([re_clustered_df, cluster_df], ignore_index=True)
            new_cluster_counter += 1

        return re_clustered_df, new_cluster_counter

    if len(single_line_clusters) > 0:
        pcodes = list(single_line_clusters['Pin code '])
        dist_matrix = get_distance_matrix(pcodes)
        re_clustered_df, new_cluster_counter = cluster_within_distance(single_line_clusters, dist_matrix, 100, new_cluster_counter)
        Sum_thick_GW = pd.concat([Sum_thick_GW, re_clustered_df], ignore_index=True)

    return Sum_thick_GW
Sum_thick_GW = clubbing_thik_material(Sum_thick_GW)
# Sum_thick_GW

In [19]:
def reassign_cluster_number_1(Sum_thick_GW):
    unique_clusters = Sum_thick_GW['Cluster_No'].unique()
    if len(unique_clusters) > 501:
        raise ValueError("Number of unique clusters exceeds the range of 500 to 1000")
    
    cluster_mapping = {old_cluster: new_cluster for old_cluster, new_cluster in zip(unique_clusters, range(500, 500 + len(unique_clusters)))}
    Sum_thick_GW['Cluster_No'] = Sum_thick_GW['Cluster_No'].map(cluster_mapping)
    
    return Sum_thick_GW
Sum_thick_GW = reassign_cluster_number_1(Sum_thick_GW)

In [20]:
def thick_vehicle_allocation(Sum_thick_GW, north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, thick_material_full):
    Sum_thick_GW['Vehicle'] = 0
    Sum_thick_GW['Final_DD'] = 0
    Sum_thick_GW['Vehicle Type'] = 0
    Sum_thick_GW['Loadability_Loss'] = 0
    
    cluster_zone = Sum_thick_GW['Zone']
    cluster_city = Sum_thick_GW['Ship-To-Party City']
    cluster_state = Sum_thick_GW['Ship-To-Party State']
    
    for i in Sum_thick_GW['Cluster_No'].unique():
        if i > 0:
            cluster_data = Sum_thick_GW[Sum_thick_GW['Cluster_No'] == i]
            weight = cluster_data['Gross Weight'].sum()
            distance = cluster_data['Distance'].max()
            FinalDD = cluster_data['Expected DispatchD'].max()
    
            zone = cluster_data['Zone'].iloc[0]
            city = cluster_data['Ship-To-Party City'].iloc[0]
            state = cluster_data['Ship-To-Party State'].iloc[0]
    
            if zone == 'NORTH EAST':
                df = north_east_thick_material_fulll
            elif zone == 'EAST':
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thick_material_full
            elif state in ['KERALA', 'TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = thick_material_full
    
            for j in range(len(df)):
                if weight <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'Final_DD'] = FinalDD
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'Vehicle'] = df.loc[j, 'Vehicle Capacity in MT']
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'Min_Weight_Kgs'] = df.loc[j, 'Min_Weight_Kgs']
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'Max_Weight_Kgs'] = df.loc[j, 'Max_Weight_Kgs']
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'Vehicle Type'] = df.loc[j, 'Vehicle Type']
                    Sum_thick_GW.loc[Sum_thick_GW['Cluster_No'] == i, 'review_limit'] = df.loc[j, 'review_limit']
                    break
    
    assert (Sum_thick_GW['Cluster_No'] > 0).all(), "All Cluster_No should be positive."
    return Sum_thick_GW
Sum_thick_GW = thick_vehicle_allocation(Sum_thick_GW, north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, thick_material_full)
# Sum_thick_GW

In [21]:
def thick_weight_approval(Sum_thick_GW):
    Sum_thick_GW['Min_Weight_Kgs'] = Sum_thick_GW['Min_Weight_Kgs'].astype('float64')
    Sum_thick_GW['Max_Weight_Kgs'] = Sum_thick_GW['Max_Weight_Kgs'].astype('float64')
    Sum_thick_GW['Total_Gross_Weight'] = Sum_thick_GW.groupby('Cluster_No')['Gross Weight'].transform('sum')
    Sum_thick_GW['Loadability_Loss'] = Sum_thick_GW['Max_Weight_Kgs'] - Sum_thick_GW['Total_Gross_Weight']
    
    approval_conditions2 = []
    
    for i in range(len(Sum_thick_GW)):
        minweight = Sum_thick_GW['Min_Weight_Kgs'][i]
        maxweight = Sum_thick_GW['Max_Weight_Kgs'][i]
        total_gross_weight = Sum_thick_GW['Total_Gross_Weight'][i]
        review_limit = Sum_thick_GW['review_limit'][i]
        
        if minweight <= total_gross_weight <= maxweight:
            approval_conditions2.append('Approved')
        elif total_gross_weight < review_limit:
            approval_conditions2.append('Reject') 
        elif minweight > total_gross_weight >= review_limit:
            approval_conditions2.append('Review') 
        else:
            approval_conditions2.append('NA')
    
    if len(approval_conditions2) == len(Sum_thick_GW):
        Sum_thick_GW['Approval Condition'] = approval_conditions2
     #print(len(Sum_thick_GW))
    return Sum_thick_GW
Sum_thick_GW = thick_weight_approval(Sum_thick_GW)

In [22]:
def thick_exceed_weight(Sum_thick_GW, thick_material_full):
    min_threshold = min(thick_material_full['Min_Threshold'])
    
    exceed_weight_thick = Sum_thick_GW[
        ((Sum_thick_GW['Total_Gross_Weight'] > min_threshold) & 
        (Sum_thick_GW['Approval Condition'] != 'Approved') & 
        (Sum_thick_GW['Approval Condition'] != 'Review')) | 
        (Sum_thick_GW['Vehicle'].isnull() | (Sum_thick_GW['Vehicle'] == 0))
    ].reset_index(drop=True)
    
    return exceed_weight_thick

exceed_weight_thick = thick_exceed_weight(Sum_thick_GW, thick_material_full)
# exceed_weight_thick

In [23]:
def thick_exceed_weight_vehicle_allocation(exceed_weight_thick,north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, thick_material_full):
    cluster_zone = exceed_weight_thick['Zone']
    cluster_city = exceed_weight_thick['Ship-To-Party City']
    cluster_state = exceed_weight_thick['Ship-To-Party State']
    exceed_weight_thick['dummy'] = exceed_weight_thick['Total_Gross_Weight']
    
    for i in range(len(exceed_weight_thick)):
        Vh = ''
        Vt = ''
        Min = ''
        Max = ''
        rev = ''
        distance = exceed_weight_thick.loc[i, 'Distance']
        
        if i in cluster_zone.index and i in cluster_city.index:
            zone = cluster_zone[i]
            city = cluster_city[i]
            state = cluster_state[i]
            
            if zone in ['NORTH EAST']:
                df = north_east_thick_material_fulll
            elif zone in ['EAST']:
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thick_material_full
            elif state in ['KERALA', 'TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = thick_material_full

        update = True
        while exceed_weight_thick['dummy'][i] > 0 and update:
            update = False
            j = len(df) - 1
            while j >= 0:
                if exceed_weight_thick['dummy'][i] >= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j, 'review_limit']) + ','
                    exceed_weight_thick.loc[i, 'dummy'] += -df.loc[j, 'Max_Weight_Kgs']
                elif exceed_weight_thick['dummy'][i] >= df.loc[j, 'Min_Weight_Kgs'] and exceed_weight_thick['dummy'][i] <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j, 'review_limit']) + ','
                    exceed_weight_thick.loc[i, 'Loadability_Loss'] = df.loc[j, 'Max_Weight_Kgs'] - exceed_weight_thick.loc[i, 'dummy']
                    exceed_weight_thick.loc[i, 'dummy'] = 0
                    update = True
                else:
                    j += -1

        exceed_weight_thick.loc[i, 'Final_DD'] = exceed_weight_thick.loc[i, 'Indent Date']
        exceed_weight_thick.loc[i, 'Vehicle'] = Vh
        exceed_weight_thick.loc[i, 'Vehicle Type'] = Vt
        exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] = Min
        exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] = Max
        exceed_weight_thick.loc[i, 'review_limit'] = rev
        
        if zone in ['NORTH EAST'] and distance > 0 and exceed_weight_thick.loc[i, 'dummy'] > 0:
            exceed_weight_thick.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_thick.loc[i, 'review_limit'] += '21375'
            exceed_weight_thick.loc[i, 'Loadability_Loss'] = 22800 - exceed_weight_thick.loc[i, 'dummy']
            exceed_weight_thick.loc[i, 'dummy'] = 0
        elif zone in ['EAST'] and distance > 0 and exceed_weight_thick.loc[i, 'dummy'] > 0:
            exceed_weight_thick.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_thick.loc[i, 'review_limit'] += '21375'
            exceed_weight_thick.loc[i, 'Loadability_Loss'] = 22800 - exceed_weight_thick.loc[i, 'dummy']
            exceed_weight_thick.loc[i, 'dummy'] = 0
        elif zone in ['SOUTH'] and state not in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_thick.loc[i, 'dummy'] > 0:
            exceed_weight_thick.loc[i, 'Vehicle'] += '24 MT'
            exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Dala'
            exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '23500'
            exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '24000'
            exceed_weight_thick.loc[i, 'review_limit'] += '22325'
            exceed_weight_thick.loc[i, 'Loadability_Loss'] = 24000 - exceed_weight_thick.loc[i, 'dummy']
        elif state in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_thick.loc[i, 'dummy'] > 0:
            exceed_weight_thick.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Domestic water Container'
            exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '23500'
            exceed_weight_thick.loc[i, 'review_limit'] += '21375'
            exceed_weight_thick.loc[i, 'Loadability_Loss'] = 23500 - exceed_weight_thick.loc[i, 'dummy']
            exceed_weight_thick.loc[i, 'dummy'] = 0
        else:
            if distance > 600 and exceed_weight_thick.loc[i, 'dummy'] > 0:
                exceed_weight_thick.loc[i, 'Vehicle'] += '18 MT'
                exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Dala'
                exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '17500'
                exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '18000'
                exceed_weight_thick.loc[i, 'review_limit'] += '16625'
                exceed_weight_thick.loc[i, 'Loadability_Loss'] = 18000 - exceed_weight_thick.loc[i, 'dummy']
                exceed_weight_thick.loc[i, 'dummy'] = 0
            elif exceed_weight_thick.loc[i, 'dummy'] > 0 and distance <= 300:
                for j in range(len(thick_material_full) - 1, -1, -1):
                    if exceed_weight_thick.loc[i, 'dummy'] <= thick_material_full['Max_Weight_Kgs'][j] and distance <= thick_material_full.loc[j, 'Max Distance']:
                        exceed_weight_thick.loc[i, 'Vehicle'] += '6 MT'
                        exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '5800'
                        exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '6200'
                        exceed_weight_thick.loc[i, 'review_limit'] += '5510'
                        exceed_weight_thick.loc[i, 'Loadability_Loss'] = 6200 - exceed_weight_thick.loc[i, 'dummy']
                        break
            elif exceed_weight_thick.loc[i, 'dummy'] > 0 and distance <= 600:
                for j in range(len(thick_material_full) - 1, -1, -1):
                    if exceed_weight_thick.loc[i, 'dummy'] <= thick_material_full['Max_Weight_Kgs'][j] and distance <= thick_material_full.loc[j, 'Max Distance']:
                        exceed_weight_thick.loc[i, 'Vehicle'] += '10 MT'
                        exceed_weight_thick.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_thick.loc[i, 'Min_Weight_Kgs'] += '9500'
                        exceed_weight_thick.loc[i, 'Max_Weight_Kgs'] += '9750'
                        exceed_weight_thick.loc[i, 'review_limit'] += '9025'
                        exceed_weight_thick.loc[i, 'Loadability_Loss'] = 9750 - exceed_weight_thick.loc[i, 'dummy']
                        break

    exceed_weight_thick['Vehicle'] = exceed_weight_thick['Vehicle'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thick['Vehicle Type'] = exceed_weight_thick['Vehicle Type'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thick['Min_Weight_Kgs'] = exceed_weight_thick['Min_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thick['Max_Weight_Kgs'] = exceed_weight_thick['Max_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    
    return exceed_weight_thick
exceed_weight_thick = thick_exceed_weight_vehicle_allocation(exceed_weight_thick,north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, thick_material_full)

In [24]:
def thick_exceed_weight_vehicle_approval(exceed_weight_thick):
    approval_conditions_3 = []
    
    for i in range(len(exceed_weight_thick)):
        minweight_str = str(exceed_weight_thick['Min_Weight_Kgs'][i])
        maxweight_str = str(exceed_weight_thick['Max_Weight_Kgs'][i])
        review_str = str(exceed_weight_thick['review_limit'][i])
        loadability_loss = exceed_weight_thick['Loadability_Loss'][i]
        
        minweight_list = [float(weight) for weight in minweight_str.split(',') if weight.strip()]
        maxweight_list = [float(weight) for weight in maxweight_str.split(',') if weight.strip()]
        review_list = [float(weight) for weight in review_str.split(',') if weight.strip()]
        
        conditions_for_vehicles = []
        
        last_min_weight = minweight_list[-1]
        last_max_weight = maxweight_list[-1]
        last_review_weight = review_list[-1]
        weight1 = last_max_weight - last_min_weight
        assign_weight = last_max_weight - loadability_loss
        
        for idx, (minweight, maxweight) in enumerate(zip(minweight_list, maxweight_list)):
            if idx < len(minweight_list) - 1:
                conditions_for_vehicles.append('Approved')
            else:
                if weight1 > loadability_loss:
                    conditions_for_vehicles.append('Approved')
                elif last_min_weight > assign_weight and assign_weight < last_review_weight:
                    conditions_for_vehicles.append('Reject')
                else:
                    conditions_for_vehicles.append('Review')
        
        approval_conditions_3.append(', '.join(conditions_for_vehicles))

    if len(approval_conditions_3) == len(exceed_weight_thick):
        exceed_weight_thick['Approval Condition'] = approval_conditions_3
    
    return exceed_weight_thick
exceed_weight_thick = thick_exceed_weight_vehicle_approval(exceed_weight_thick)


In [25]:
def thick_exceed_weight_vehicle_approval(exceed_weight_thick):
    approval_conditions_3 = []
    
    for i in range(len(exceed_weight_thick)):
        minweight_str = str(exceed_weight_thick['Min_Weight_Kgs'][i])
        maxweight_str = str(exceed_weight_thick['Max_Weight_Kgs'][i])
        review_str = str(exceed_weight_thick['review_limit'][i])
        loadability_loss = exceed_weight_thick['Loadability_Loss'][i]
        
        minweight_list = [float(weight) for weight in minweight_str.split(',') if weight.strip()]
        maxweight_list = [float(weight) for weight in maxweight_str.split(',') if weight.strip()]
        review_list = [float(weight) for weight in review_str.split(',') if weight.strip()]
        
        conditions_for_vehicles = []
        
        last_min_weight = minweight_list[-1]
        last_max_weight = maxweight_list[-1]
        last_review_weight = review_list[-1]
        weight1 = last_max_weight - last_min_weight
        assign_weight = last_max_weight - loadability_loss
        
        for idx, (minweight, maxweight) in enumerate(zip(minweight_list, maxweight_list)):
            if idx < len(minweight_list) - 1:
                conditions_for_vehicles.append('Approved')
            else:
                if weight1 > loadability_loss:
                    conditions_for_vehicles.append('Approved')
                elif last_min_weight > assign_weight and assign_weight < last_review_weight:
                    conditions_for_vehicles.append('Reject')
                else:
                    conditions_for_vehicles.append('Review')
        
        approval_conditions_3.append(', '.join(conditions_for_vehicles))

    if len(approval_conditions_3) == len(exceed_weight_thick):
        exceed_weight_thick['Approval Condition'] = approval_conditions_3
    
    return exceed_weight_thick

In [26]:
def update_loadability_and_vehicle_thick(Sum_thick_GW, exceed_weight_thick):
    for i in Sum_thick_GW.index:
        for j in exceed_weight_thick.index:
            if Sum_thick_GW.loc[i, 'Cluster_No'] == exceed_weight_thick.loc[j, 'Cluster_No']:
                Sum_thick_GW.loc[i, 'Loadability_Loss'] = exceed_weight_thick.loc[j, 'Loadability_Loss']
                Sum_thick_GW.loc[i, 'Vehicle'] = exceed_weight_thick.loc[j, 'Vehicle']
                Sum_thick_GW.loc[i, 'Vehicle Type'] = exceed_weight_thick.loc[j, 'Vehicle Type']
                Sum_thick_GW.loc[i, 'Final_DD'] = exceed_weight_thick.loc[j, 'Expected DispatchD']
                Sum_thick_GW.loc[i, 'Max_Weight_Kgs'] = exceed_weight_thick.loc[j, 'Max_Weight_Kgs']
                Sum_thick_GW.loc[i, 'Min_Weight_Kgs'] = exceed_weight_thick.loc[j, 'Min_Weight_Kgs']
                Sum_thick_GW.loc[i, 'Approval Condition'] = exceed_weight_thick.loc[j, 'Approval Condition']
                Sum_thick_GW.loc[i, 'review_limit'] = exceed_weight_thick.loc[j, 'review_limit']

    Approved2 = Sum_thick_GW[Sum_thick_GW['Approval Condition'].str.startswith('Approved') | (Sum_thick_GW['Approval Condition'] == 'Review')].reset_index()
    reject_review2 = Sum_thick_GW[Sum_thick_GW['Approval Condition'].isin(['Reject'])].reset_index()
    return Approved2, reject_review2
Approved2, reject_review2 = update_loadability_and_vehicle_thick(Sum_thick_GW, exceed_weight_thick)

In [27]:
def thin_vehicle_allocation(Sum_thin_GW, north_east_thick_material_fulll, east_mixed_material, south_thin_material_full, kerala_tn_material, thin_material_full):
    Sum_thin_GW['Vehicle'] = 0
    Sum_thin_GW['Final_DD'] = 0
    Sum_thin_GW['Vehicle Type'] = 0
    Sum_thin_GW['Loadability_Loss'] = 0
    cluster_zone = Sum_thin_GW['Zone']
    cluster_city = Sum_thin_GW['Ship-To-Party City']
    cluster_state = Sum_thin_GW['Ship-To-Party State']
    
    for i in Sum_thin_GW['Cluster_No'].unique():
        if i > 0: 
            cluster_data = Sum_thin_GW[Sum_thin_GW['Cluster_No'] == i]
            weight = cluster_data['Gross Weight'].sum()
            distance = cluster_data['Distance'].max()
            FinalDD = cluster_data['Expected DispatchD'].max()

            zone = cluster_data['Zone'].iloc[0]
            city = cluster_data['Ship-To-Party City'].iloc[0]
            state = cluster_data['Ship-To-Party State'].iloc[0]
            
            if zone == 'NORTH EAST':
                df = north_east_thick_material_fulll
            elif zone == 'EAST':
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thin_material_full
            elif state in ['KERALA', 'TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = thin_material_full

            for j in range(len(df)):
                if weight <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'Final_DD'] = FinalDD
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'Vehicle'] = df.loc[j, 'Vehicle Capacity in MT']
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'Min_Weight_Kgs'] = df.loc[j, 'Min_Weight_Kgs']
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'Max_Weight_Kgs'] = df.loc[j, 'Max_Weight_Kgs']
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'Vehicle Type'] = df.loc[j, 'Vehicle Type']
                    Sum_thin_GW.loc[Sum_thin_GW['Cluster_No'] == i, 'review_limit'] = df.loc[j, 'review_limit']
                    break
    
    assert (Sum_thin_GW['Cluster_No'] > 0).all(), "All Cluster_No should be positive."
    return Sum_thin_GW
Sum_thin_GW= thin_vehicle_allocation(Sum_thin_GW, north_east_thick_material_fulll, east_mixed_material, south_thin_material_full, kerala_tn_material, thin_material_full)

In [28]:
def thin_weight_approval(Sum_thin_GW):
    Sum_thin_GW['Total_Gross_Weight'] = Sum_thin_GW.groupby('Cluster_No')['Gross Weight'].transform('sum')
    Sum_thin_GW['Loadability_Loss'] = Sum_thin_GW['Max_Weight_Kgs'] - Sum_thin_GW['Total_Gross_Weight']
    approval_conditions4 = []

    for i in range(len(Sum_thin_GW)):
        minweight = Sum_thin_GW['Min_Weight_Kgs'][i]
        maxweight = Sum_thin_GW['Max_Weight_Kgs'][i]
        total_gross_weight = Sum_thin_GW['Total_Gross_Weight'][i]
        review_limit = Sum_thin_GW['review_limit'][i]

        if minweight <= total_gross_weight <= maxweight:
            approval_conditions4.append('Approved')
        elif total_gross_weight < review_limit:
            approval_conditions4.append('Reject')
        elif minweight > total_gross_weight >= review_limit:
            approval_conditions4.append('Review')
        else:
            approval_conditions4.append('NA')

    if len(approval_conditions4) == len(Sum_thin_GW):
        Sum_thin_GW['Approval Condition'] = approval_conditions4

    return Sum_thin_GW

Sum_thin_GW = thin_weight_approval(Sum_thin_GW)

In [29]:
def thin_exceed_weight(Sum_thin_GW, thin_material_full):
    min_threshold = min(thin_material_full['Min_Threshold'])
    Sum_thin_GW['Total_Gross_Weight'] = Sum_thin_GW.groupby('Cluster_No')['Gross Weight'].transform('sum')

    exceed_weight_thin = Sum_thin_GW[
        ((Sum_thin_GW['Total_Gross_Weight'] > min_threshold) & 
        (Sum_thin_GW['Approval Condition'] == 'Reject')) |
        (Sum_thin_GW['Vehicle'].isnull() | (Sum_thin_GW['Vehicle'] == 0))
    ].reset_index(drop=True)

    return exceed_weight_thin
exceed_weight_thin = thin_exceed_weight(Sum_thin_GW, thin_material_full)

In [30]:
def thin_exceed_weight_vehicle_allocation(exceed_weight_thin, north_east_thick_material_fulll, east_mixed_material, south_thin_material_full, kerala_tn_material, thin_material_full):
    cluster_zone = exceed_weight_thin['Zone']
    cluster_city = exceed_weight_thin['Ship-To-Party City']
    cluster_state = exceed_weight_thin['Ship-To-Party State']
    exceed_weight_thin['dummy'] = exceed_weight_thin['Total_Gross_Weight']
    
    for i in range(len(exceed_weight_thin)):
        Vh = ''
        Vt = ''
        Min = ''
        Max = ''
        rev = ''
        distance = exceed_weight_thin.loc[i, 'Distance']
        
        if i in cluster_zone.index and i in cluster_city.index:
            zone = cluster_zone[i]
            city = cluster_city[i]
            state = cluster_state[i]
            
            if zone in ['NORTH EAST']:
                df = north_east_thick_material_fulll
            elif zone in ['EAST']:
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thin_material_full
            elif state in ['KERALA', 'TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = thin_material_full
                
        update = True
        while exceed_weight_thin['dummy'][i] > 0 and update:
            update = False
            j = len(df) - 1
            
            while j >= 0:
                if exceed_weight_thin['dummy'][i] >= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j, 'review_limit']) + ','
                    exceed_weight_thin.loc[i, 'dummy'] += -df.loc[j, 'Max_Weight_Kgs']
                elif exceed_weight_thin['dummy'][i] >= df.loc[j, 'Min_Weight_Kgs'] and exceed_weight_thin['dummy'][i] <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j, 'review_limit']) + ','
                    exceed_weight_thin.loc[i, 'Loadability_Loss'] = df.loc[j, 'Max_Weight_Kgs'] - exceed_weight_thin.loc[i, 'dummy']
                    exceed_weight_thin.loc[i, 'dummy'] = 0
                    update = True
                else:
                    j += -1
        
        exceed_weight_thin.loc[i, 'Final_DD'] = exceed_weight_thin.loc[i, 'Indent Date']
        exceed_weight_thin.loc[i, 'Vehicle'] = Vh
        exceed_weight_thin.loc[i, 'Vehicle Type'] = Vt
        exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] = Min
        exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] = Max
        exceed_weight_thin.loc[i, 'review_limit'] = rev
        
        if zone in ['NORTH EAST'] and distance > 0 and exceed_weight_thin.loc[i, 'dummy'] > 0:
            exceed_weight_thin.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_thin.loc[i, 'review_limit'] += '21375'
            exceed_weight_thin.loc[i, 'Loadability_Loss'] = 22800 - exceed_weight_thin.loc[i, 'dummy']
            exceed_weight_thin.loc[i, 'dummy'] = 0
        elif zone in ['EAST'] and distance > 0 and exceed_weight_thin.loc[i, 'dummy'] > 0:
            exceed_weight_thin.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_thin.loc[i, 'review_limit'] += '21375'
            exceed_weight_thin.loc[i, 'Loadability_Loss'] = 22800 - exceed_weight_thin.loc[i, 'dummy']
            exceed_weight_thin.loc[i, 'dummy'] = 0
        elif zone in ['SOUTH'] and state not in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_thin.loc[i, 'dummy'] > 0:
            exceed_weight_thin.loc[i, 'Vehicle'] += '24 MT'
            exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '23000'
            exceed_weight_thin.loc[i, 'review_limit'] += '21375'
            exceed_weight_thin.loc[i, 'Loadability_Loss'] = 23000 - exceed_weight_thin.loc[i, 'dummy']
            exceed_weight_thin.loc[i, 'dummy'] = 0
        elif state in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_thin.loc[i, 'dummy'] > 0:
            exceed_weight_thin.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Domestic water Container'
            exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '23500'
            exceed_weight_thin.loc[i, 'review_limit'] += '21375'
            exceed_weight_thin.loc[i, 'Loadability_Loss'] = 23500 - exceed_weight_thin.loc[i, 'dummy']
            exceed_weight_thin.loc[i, 'dummy'] = 0
        else:
            if distance > 600 and exceed_weight_thin.loc[i, 'dummy'] > 0:
                exceed_weight_thin.loc[i, 'Vehicle'] += '18 MT'
                exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Container'
                exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '16800'
                exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '17000'
                exceed_weight_thin.loc[i, 'review_limit'] += '15960'
                exceed_weight_thin.loc[i, 'Loadability_Loss'] = 17000 - exceed_weight_thin.loc[i, 'dummy']
                exceed_weight_thin.loc[i, 'dummy'] = 0
            elif exceed_weight_thin.loc[i, 'dummy'] > 0 and distance <= 300:
                for j in range(len(thin_material_full) - 1, -1, -1):
                    if exceed_weight_thin.loc[i, 'dummy'] <= thin_material_full['Max_Weight_Kgs'][j] and distance <= thin_material_full.loc[j, 'Max Distance']:
                        exceed_weight_thin.loc[i, 'Vehicle'] += '6 MT'
                        exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '5800'
                        exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '6200'
                        exceed_weight_thin.loc[i, 'review_limit'] += '5510'
                        exceed_weight_thin.loc[i, 'Loadability_Loss'] = 6200 - exceed_weight_thin.loc[i, 'dummy']
                        break
            elif exceed_weight_thin.loc[i, 'dummy'] > 0 and distance <= 600:
                for j in range(len(thin_material_full) - 1, -1, -1):
                    if exceed_weight_thin.loc[i, 'dummy'] <= thin_material_full['Max_Weight_Kgs'][j] and distance <= thin_material_full.loc[j, 'Max Distance']:
                        exceed_weight_thin.loc[i, 'Vehicle'] += '10 MT'
                        exceed_weight_thin.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_thin.loc[i, 'Min_Weight_Kgs'] += '9500'
                        exceed_weight_thin.loc[i, 'Max_Weight_Kgs'] += '9750'
                        exceed_weight_thin.loc[i, 'review_limit'] += '9025'
                        exceed_weight_thin.loc[i, 'Loadability_Loss'] = 9750 - exceed_weight_thin.loc[i, 'dummy']
                        break

    exceed_weight_thin['Vehicle'] = exceed_weight_thin['Vehicle'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thin['Vehicle Type'] = exceed_weight_thin['Vehicle Type'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thin['Min_Weight_Kgs'] = exceed_weight_thin['Min_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thin['Max_Weight_Kgs'] = exceed_weight_thin['Max_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_thin['review_limit'] = exceed_weight_thin['review_limit'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)

    return exceed_weight_thin
exceed_weight_thin = thin_exceed_weight_vehicle_allocation(exceed_weight_thin, north_east_thick_material_fulll, east_mixed_material, south_thin_material_full, kerala_tn_material, thin_material_full)

In [31]:
def thin_exceed_weight_vehicle_approval(exceed_weight_thin):
    approval_conditions_5 = []
    for i in range(len(exceed_weight_thin)):
        minweight_str = str(exceed_weight_thin['Min_Weight_Kgs'][i])
        maxweight_str = str(exceed_weight_thin['Max_Weight_Kgs'][i])
        review_str = str(exceed_weight_thin['review_limit'][i])
        loadability_loss = exceed_weight_thin['Loadability_Loss'][i]
        
        minweight_list = [float(weight) for weight in minweight_str.split(',') if weight.strip()]
        maxweight_list = [float(weight) for weight in maxweight_str.split(',') if weight.strip()]
        review_list = [float(weight) for weight in review_str.split(',') if weight.strip()]
        
        conditions_for_vehicles = []
        last_min_weight = minweight_list[-1]
        last_max_weight = maxweight_list[-1]
        last_review_weight = review_list[-1]
        weight1 = last_max_weight - last_min_weight
        assign_weight = last_max_weight - loadability_loss
        
        for idx, (minweight, maxweight) in enumerate(zip(minweight_list, maxweight_list)):
            if idx < len(minweight_list) - 1:
                conditions_for_vehicles.append('Approved')
            else:
                if weight1 > loadability_loss:
                    conditions_for_vehicles.append('Approved')
                elif last_min_weight > assign_weight and assign_weight < last_review_weight:
                    conditions_for_vehicles.append('Reject')
                else:
                    conditions_for_vehicles.append('Review')
        
        approval_conditions_5.append(', '.join(conditions_for_vehicles))
    
    if len(approval_conditions_5) == len(exceed_weight_thin):
        exceed_weight_thin['Approval Condition'] = approval_conditions_5
    return exceed_weight_thin
exceed_weight_thin = thin_exceed_weight_vehicle_approval(exceed_weight_thin)

In [32]:
def update_loadability_and_vehicle_thin(Sum_thin_GW, exceed_weight_thin):
    for i in Sum_thin_GW.index:
        for j in exceed_weight_thin.index:
            if Sum_thin_GW.loc[i, 'Cluster_No'] == exceed_weight_thin.loc[j, 'Cluster_No']:
                Sum_thin_GW.loc[i, 'Loadability_Loss'] = exceed_weight_thin.loc[j, 'Loadability_Loss']
                Sum_thin_GW.loc[i, 'Vehicle'] = exceed_weight_thin.loc[j, 'Vehicle']
                Sum_thin_GW.loc[i, 'Vehicle Type'] = exceed_weight_thin.loc[j, 'Vehicle Type']
                Sum_thin_GW.loc[i, 'Final_DD'] = exceed_weight_thin.loc[j, 'Expected DispatchD']
                Sum_thin_GW.loc[i, 'Max_Weight_Kgs'] = exceed_weight_thin.loc[j, 'Max_Weight_Kgs']
                Sum_thin_GW.loc[i, 'Min_Weight_Kgs'] = exceed_weight_thin.loc[j, 'Min_Weight_Kgs']
                Sum_thin_GW.loc[i, 'Approval Condition'] = exceed_weight_thin.loc[j, 'Approval Condition']
    
    Approved1 = Sum_thin_GW[Sum_thin_GW['Approval Condition'].str.startswith('Approved') | (Sum_thin_GW['Approval Condition'] == 'Review')].reset_index()
    reject_review1 = Sum_thin_GW[Sum_thin_GW['Approval Condition'].isin(['Reject'])].reset_index()
    return Approved1, reject_review1
Approved1,reject_review1 = update_loadability_and_vehicle_thin(Sum_thin_GW, exceed_weight_thin)

In [33]:
Sum_mixed_GW

,Indent No.,Indent Date,todayDate,Ship-To-Party,Ship-To-Party Name,Ship-To-Party City,Ship-To-Party State,Zone,Pin code,Sales Office,Distance,TAT,Gross Weight,Order CBM,Expected DispatchD
0,1530012901,2024-10-07,2024-11-06,1000029557,RAVI AGENCY,HYDERABAD,TELANGANA,SOUTH,500001,SHYD,1170.0,5.0,1054.40,4.495,2024-11-06
1,1530012921,2024-10-08,2024-11-06,1000001172,TODI PLYWOOD,KOLKATA,WEST BENGAL,EAST,700012,EKOL,1960.0,8.0,3643.60,32.567,2024-11-06
2,1530012953,2024-10-08,2024-11-06,1000028745,ZEENATH PLYWOODS & CORPORATE INTERIOR,MANGALORE,KARNATAKA,SOUTH,575001,SBLR,1388.0,6.0,278.34,1.607,2024-11-06
3,1530012955,2024-10-08,2024-11-06,1000028745,ZEENATH PLYWOODS & CORPORATE INTERIOR,MANGALORE,KARNATAKA,SOUTH,575001,SBLR,1388.0,6.0,1194.62,1.438,2024-11-06
4,1530013016,2024-10-09,2024-11-06,1000028767,SHIVANGI INDUSTRIES,JAIPUR,RAJASTHAN,NORTH,303007,NJPR,626.0,3.0,0.00,24.029,2024-11-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,1530013717,2024-11-02,2024-11-06,1000029060,A B M WOOD DECOR PRIVATE LIMITED,SURAT,GUJARAT,WEST,394510,WSRT,246.0,2.0,0.00,23.360,2024-11-06
99,1530013720,2024-11-04,2024-11-06,1000029307,MAHESH PLYWOOD,AJMER,RAJASTHAN,NORTH,305001,NJPR,514.0,3.0,9473.92,12.782,2024-11-06
100,1530013722,2024-11-04,2024-11-06,1000002022,VISHWANATH GLASS PLY & HARDWARE,VARANASI,UTTAR PRADESH,CENTRA,221005,NLKW,1295.0,5.0,12549.43,14.690,2024-11-06
101,1530013726,2024-11-04,2024-11-06,1000029278,GLO PANELS PRIVATE LIMITED,MUMBAI,MAHARASHTRA,WEST,421601,WMUM,507.0,3.0,7514.91,11.952,2024-11-06


In [34]:
def clubbing_mixed_material(Sum_mixed_GW):
    newclub = Sum_mixed_GW
    newclub = newclub.groupby(['Indent No.','Indent Date','todayDate','Ship-To-Party','Ship-To-Party Name','Ship-To-Party City','Ship-To-Party State','Zone','Pin code ','Sales Office','Distance','TAT']).agg({'Gross Weight': 'sum', 'Order CBM': 'sum', 'Expected DispatchD': 'max'}).reset_index()
    newclub['Cluster_No'] = -1 
    newclub['Loading_order'] = 0 
    newclub['Min_Weight_Kgs'] = 0 
    newclub['Max_Weight_Kgs'] = 0 
    newclub['Cluster_No'] = newclub['Indent No.']

    new_cluster_counter = newclub['Cluster_No'].max() + 1
    for city, group in newclub.groupby('Ship-To-Party City'):
        party_names = group['Ship-To-Party Name'].unique()
        for i in range(0, len(party_names), 2):
            names_subset = party_names[i:i+2]
            subset_group = group[group['Ship-To-Party Name'].isin(names_subset)]
            newclub.loc[subset_group.index, 'Cluster_No'] = new_cluster_counter
            new_cluster_counter += 1

    single_line_clusters = newclub[newclub.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    newclub = newclub[~newclub['Cluster_No'].isin(single_line_clusters['Cluster_No'])]

    new_cluster_counter = newclub['Cluster_No'].max() + 1
    re_clustered_df = pd.DataFrame()
    for city, group in single_line_clusters.groupby('Ship-To-Party City'):
        group = group.sort_values(by='Indent No.').reset_index(drop=True)
        clusters = np.array_split(group, np.ceil(len(group) / 2))
        for cl in clusters:
            if len(cl) > 0:
                cl['Cluster_No'] = new_cluster_counter
                re_clustered_df = pd.concat([re_clustered_df, cl], ignore_index=True)
                new_cluster_counter += 1

    newclub = pd.concat([newclub, re_clustered_df], ignore_index=True)

    single_line_clusters = newclub[newclub.groupby('Cluster_No')['Cluster_No'].transform('size') == 1]
    newclub = newclub[~newclub['Cluster_No'].isin(single_line_clusters['Cluster_No'])]

    def get_distance_matrix(postal_codes):
        india = pgeocode.GeoDistance('IN')
        dist_mat = pd.DataFrame(index=postal_codes, columns=postal_codes)
        for i in range(len(postal_codes)):
            for j in range(i, len(postal_codes)):
                if i == j:
                    dist_mat.iloc[i, j] = 0
                else:
                    dist = india.query_postal_code(postal_codes[i], postal_codes[j])
                    dist_mat.iloc[i, j] = dist
                    dist_mat.iloc[j, i] = dist
        return dist_mat

    def cluster_within_distance(df, dist_matrix, max_distance, new_cluster_counter):
        indices_used = set()
        re_clustered_df = pd.DataFrame()
        for i in range(len(dist_matrix)):
            if i in indices_used:
                continue
            indices_used.add(i)
            cluster_indices = [i]
            for j in range(i + 1, len(dist_matrix)):
                if j in indices_used:
                    continue
                if dist_matrix.iloc[i, j] <= max_distance and len(cluster_indices) < 2:
                    cluster_indices.append(j)
                    indices_used.add(j)
            cluster_df = df.iloc[cluster_indices].copy()
            cluster_df['Cluster_No'] = new_cluster_counter
            re_clustered_df = pd.concat([re_clustered_df, cluster_df], ignore_index=True)
            new_cluster_counter += 1
        return re_clustered_df, new_cluster_counter

    if len(single_line_clusters) > 0:
        pcodes = list(single_line_clusters['Pin code '])
        dist_matrix = get_distance_matrix(pcodes)
        re_clustered_df, new_cluster_counter = cluster_within_distance(single_line_clusters, dist_matrix, 100, new_cluster_counter)
        newclub = pd.concat([newclub, re_clustered_df], ignore_index=True)

    return newclub
newclub = clubbing_mixed_material(Sum_mixed_GW)

In [35]:
def reassign_cluster_number_2(newclub):
    unique_clusters = newclub['Cluster_No'].unique()
    if len(unique_clusters) > 501:
        raise ValueError("Number of unique clusters exceeds the range of 1000 to 1500")

    cluster_mapping = {old_cluster: new_cluster for old_cluster, new_cluster in zip(unique_clusters, range(1000, 1000 + len(unique_clusters)))}
    newclub['Cluster_No'] = newclub['Cluster_No'].map(cluster_mapping)

    return newclub
newclub = reassign_cluster_number_2(newclub)

In [36]:
def mixed_vehicle_allocation(newclub, north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, mixed_material):
    newclub['Vehicle'] = 0
    newclub['Final_DD'] = 0
    newclub['Vehicle Type'] = 0
    newclub['Loadability_Loss'] = 0
    
    cluster_zone = newclub['Zone']
    cluster_city = newclub['Ship-To-Party City']
    cluster_state = newclub['Ship-To-Party State']
    for i in newclub['Cluster_No'].unique():
        if i > 0:
            cluster_data = newclub[newclub['Cluster_No'] == i]
            weight = cluster_data['Gross Weight'].sum()
            distance = cluster_data['Distance'].max()
            FinalDD = cluster_data['Expected DispatchD'].max()

            zone = cluster_data['Zone'].iloc[0]
            city = cluster_data['Ship-To-Party City'].iloc[0]
            state = cluster_data['Ship-To-Party State'].iloc[0]

            if zone == 'NORTH EAST':
                df = north_east_thick_material_fulll
            elif zone == 'EAST':
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thick_material_full
            elif state in ['KERALA', 'TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = mixed_material

            for j in range(len(df)):
                if weight <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    newclub.loc[newclub['Cluster_No'] == i, 'Final_DD'] = FinalDD
                    newclub.loc[newclub['Cluster_No'] == i, 'Vehicle'] = df.loc[j, 'Vehicle Capacity in MT']
                    newclub.loc[newclub['Cluster_No'] == i, 'Min_Weight_Kgs'] = df.loc[j, 'Min_Weight_Kgs']
                    newclub.loc[newclub['Cluster_No'] == i, 'Max_Weight_Kgs'] = df.loc[j, 'Max_Weight_Kgs']
                    newclub.loc[newclub['Cluster_No'] == i, 'Vehicle Type'] = df.loc[j, 'Vehicle Type']
                    newclub.loc[newclub['Cluster_No'] == i, 'review_limit'] = df.loc[j, 'review_limit']
                    break

    assert (newclub['Cluster_No'] > 0).all(), "All Cluster_No should be positive."

    return newclub
newclub = mixed_vehicle_allocation(newclub, north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material, mixed_material)

In [37]:
def mixed_weight_approval(newclub):
    newclub['Total_Gross_Weight'] = newclub.groupby('Cluster_No')['Gross Weight'].transform('sum')
    newclub['Loadability_Loss'] = newclub['Max_Weight_Kgs'] - newclub['Total_Gross_Weight']
    approval_conditions1000 = []

    for i in range(len(newclub)):
        weight1 = sum(newclub[newclub['Cluster_No'] == newclub['Cluster_No'][i]]['Gross Weight'])
        minweight = newclub['Min_Weight_Kgs'][i]
        maxweight = newclub['Max_Weight_Kgs'][i]
        total_gross_weight = newclub['Total_Gross_Weight'][i]
        review_limit = newclub['review_limit'][i]

        if minweight <= total_gross_weight <= maxweight:
            approval_conditions1000.append('Approved')
        elif total_gross_weight < review_limit:
            approval_conditions1000.append('Reject')
        elif minweight > total_gross_weight >= review_limit:
            approval_conditions1000.append('Review')
        else:
            approval_conditions1000.append('NA')

    if len(approval_conditions1000) == len(newclub):
        newclub['Approval Condition'] = approval_conditions1000

    return newclub
newclub = mixed_weight_approval(newclub)

In [38]:
def mixed_exceed_weight(newclub, mixed_material):
    min_threshold = min(mixed_material['Min_Threshold'])
    newclub['Total_Gross_Weight'] = newclub.groupby('Cluster_No')['Gross Weight'].transform('sum')

    exceed_weight_mix = newclub[
        ((newclub['Total_Gross_Weight'] > min_threshold) & 
        (newclub['Approval Condition'] == 'Reject')) |
        (newclub['Vehicle'].isnull() | (newclub['Vehicle'] == 0))
    ].reset_index(drop=True)
    
    return exceed_weight_mix
exceed_weight_mix = mixed_exceed_weight(newclub, mixed_material)

In [39]:
def mixed_exceed_weight_vehicle_allocation(exceed_weight_mix,north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material,mixed_material):
    cluster_city = exceed_weight_mix['Ship-To-Party City']
    cluster_zone = exceed_weight_mix['Zone']
    cluster_state = exceed_weight_mix['Ship-To-Party State']
    
    exceed_weight_mix['dummy'] = exceed_weight_mix['Total_Gross_Weight']
    
    for i in exceed_weight_mix.index:
        Vh = ''
        Vt = ''
        Min = ''
        Max = ''
        rev = ''
        distance = exceed_weight_mix.loc[i, 'Distance']
        
        if i in cluster_zone.index and i in cluster_city.index:
            zone = cluster_zone.loc[i]
            city = cluster_city[i]
            state = cluster_state[i]
            
            if zone in ['NORTH EAST']:
                df = north_east_thick_material_fulll
            elif zone in ['EAST']:
                df = east_mixed_material
            elif zone == 'SOUTH' and state not in ['KERALA', 'TAMIL NADU']:
                df = south_thick_material_full
            elif state in ['KERALA','TAMIL NADU']:
                df = kerala_tn_material
            else:
                df = mixed_material

        update = True
        while exceed_weight_mix['dummy'][i] > 0 and update:
            update = False
            j = len(df) - 1
            while j >= 0:
                if exceed_weight_mix['dummy'][i] >= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j,'review_limit']) + ','
                    exceed_weight_mix.loc[i, 'dummy'] += -df.loc[j, 'Max_Weight_Kgs']
                elif exceed_weight_mix['dummy'][i] >= df.loc[j, 'Min_Weight_Kgs'] and exceed_weight_mix['dummy'][i] <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Vh += str(df.loc[j, 'Vehicle Capacity in MT']) + ','
                    Vt += str(df.loc[j, 'Vehicle Type']) + ','
                    Min += str(df.loc[j, 'Min_Weight_Kgs']) + ','
                    Max += str(df.loc[j, 'Max_Weight_Kgs']) + ','
                    rev += str(df.loc[j,'review_limit']) + ','
                    exceed_weight_mix.loc[i, 'Loadability_Loss'] = df.loc[j, 'Max_Weight_Kgs'] - exceed_weight_mix.loc[i, 'dummy']
                    exceed_weight_mix.loc[i, 'dummy'] = 0
                    update = True
                else:
                    j -= 1

        exceed_weight_mix.loc[i, 'Final_DD'] = exceed_weight_mix.loc[i, 'Indent Date']
        exceed_weight_mix.loc[i, 'Vehicle'] = Vh
        exceed_weight_mix.loc[i, 'Vehicle Type'] = Vt
        exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] = Min
        exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] = Max
        exceed_weight_mix.loc[i, 'review_limit'] = rev
        if zone in ['NORTH EAST'] and distance > 0 and exceed_weight_mix.loc[i,'dummy'] > 0:
            exceed_weight_mix.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_mix.loc[i, 'review_limit'] += '21375'
            exceed_weight_mix.loc[i,'Loadability_Loss'] = 22800 - exceed_weight_mix.loc[i,'dummy']
            exceed_weight_mix.loc[i,'dummy'] = 0
        elif zone in ['EAST'] and distance > 0 and exceed_weight_mix.loc[i,'dummy'] > 0:
            exceed_weight_mix.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Container'
            exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '22800'
            exceed_weight_mix.loc[i, 'review_limit'] += '21375'
            exceed_weight_mix.loc[i,'Loadability_Loss'] = 22800 - exceed_weight_mix.loc[i,'dummy']
            exceed_weight_mix.loc[i,'dummy'] = 0
        elif zone in ['SOUTH'] and state not in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_mix.loc[i, 'dummy'] > 0:
            exceed_weight_mix.loc[i, 'Vehicle'] += '24 MT'
            exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Dala'
            exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '23500'
            exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '24000'
            exceed_weight_mix.loc[i, 'review_limit'] += '22325'
            exceed_weight_mix.loc[i, 'Loadability_Loss'] = 24000 - exceed_weight_mix.loc[i, 'dummy']
            exceed_weight_mix.loc[i, 'dummy'] = 0
        elif state in ['KERALA', 'TAMIL NADU'] and distance > 0 and exceed_weight_mix.loc[i, 'dummy'] > 0:
            exceed_weight_mix.loc[i, 'Vehicle'] += '23 MT'
            exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Domestic water Container'
            exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '22500'
            exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '23500'
            exceed_weight_mix.loc[i, 'review_limit'] += '21375'
            exceed_weight_mix.loc[i, 'Loadability_Loss'] = 23500 - exceed_weight_mix.loc[i, 'dummy']
            exceed_weight_mix.loc[i, 'dummy'] = 0
        else:
            if distance > 600 and exceed_weight_mix.loc[i,'dummy'] > 0:
                exceed_weight_mix.loc[i, 'Vehicle'] += '18 MT'
                exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Dala'
                exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '18000'
                exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '18400'
                exceed_weight_mix.loc[i, 'review_limit'] += '17100'
                exceed_weight_mix.loc[i,'Loadability_Loss'] = 18400 - exceed_weight_mix.loc[i,'dummy']
                exceed_weight_mix.loc[i,'dummy'] = 0
            elif exceed_weight_mix.loc[i, 'dummy'] > 0 and distance <= 300:
                for j in range(len(mixed_material) - 1, -1, -1):
                    if exceed_weight_mix.loc[i, 'dummy'] <= mixed_material['Max_Weight_Kgs'][j] and distance <= mixed_material.loc[j, 'Max Distance']:
                        exceed_weight_mix.loc[i, 'Vehicle'] += '6 MT'
                        exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '5800'
                        exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '6200'
                        exceed_weight_mix.loc[i, 'review_limit'] += '5510'
                        exceed_weight_mix.loc[i,'Loadability_Loss'] = 6200 - exceed_weight_mix.loc[i,'dummy']
                        break
            elif exceed_weight_mix.loc[i, 'dummy'] > 0 and distance <= 600:
                for j in range(len(mixed_material) - 1, -1, -1):
                    if exceed_weight_mix.loc[i, 'dummy'] <= mixed_material['Max_Weight_Kgs'][j] and distance <= mixed_material.loc[j, 'Max Distance']:
                        exceed_weight_mix.loc[i, 'Vehicle'] += '10 MT'
                        exceed_weight_mix.loc[i, 'Vehicle Type'] += 'Dala'
                        exceed_weight_mix.loc[i, 'Min_Weight_Kgs'] += '9500'
                        exceed_weight_mix.loc[i, 'Max_Weight_Kgs'] += '9750'
                        exceed_weight_mix.loc[i, 'review_limit'] += '9025'
                        exceed_weight_mix.loc[i,'Loadability_Loss'] = 9750 - exceed_weight_mix.loc[i,'dummy']
                        break

    exceed_weight_mix['Vehicle'] = exceed_weight_mix['Vehicle'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_mix['Vehicle Type'] = exceed_weight_mix['Vehicle Type'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_mix['Min_Weight_Kgs'] = exceed_weight_mix['Min_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)
    exceed_weight_mix['Max_Weight_Kgs'] = exceed_weight_mix['Max_Weight_Kgs'].apply(lambda x: x.rstrip(',') if isinstance(x, str) else x)

    return exceed_weight_mix
exceed_weight_mix = mixed_exceed_weight_vehicle_allocation(exceed_weight_mix,north_east_thick_material_fulll, east_mixed_material, south_thick_material_full, kerala_tn_material,mixed_material)

In [40]:
def mixed_exceed_weight_vehicle_approval(exceed_weight_mix):
    approval_conditions_11 = []

    for i in range(len(exceed_weight_mix)):
        minweight_str = str(exceed_weight_mix['Min_Weight_Kgs'][i])
        maxweight_str = str(exceed_weight_mix['Max_Weight_Kgs'][i])
        review_str = str(exceed_weight_mix['review_limit'][i])
        loadability_loss = exceed_weight_mix['Loadability_Loss'][i]

        minweight_list = [float(weight) for weight in minweight_str.split(',') if weight.strip()]
        maxweight_list = [float(weight) for weight in maxweight_str.split(',') if weight.strip()]
        review_list = [float(weight) for weight in review_str.split(',') if weight.strip()]

        conditions_for_vehicles = []

        last_min_weight = minweight_list[-1]
        last_max_weight = maxweight_list[-1]
        last_review_weight = review_list[-1]
        weight1 = last_max_weight - last_min_weight
        assign_weight = last_max_weight - loadability_loss

        for idx, (minweight, maxweight) in enumerate(zip(minweight_list, maxweight_list)):
            if idx < len(minweight_list) - 1:
                conditions_for_vehicles.append('Approved')
            else:
                if weight1 > loadability_loss:
                    conditions_for_vehicles.append('Approved')
                elif last_min_weight > assign_weight and assign_weight < last_review_weight:
                    conditions_for_vehicles.append('Reject')
                else:
                    conditions_for_vehicles.append('Review')

        approval_conditions_11.append(', '.join(conditions_for_vehicles))

    if len(approval_conditions_11) == len(exceed_weight_mix):
        exceed_weight_mix['Approval Condition'] = approval_conditions_11

    return exceed_weight_mix
exceed_weight_mix = mixed_exceed_weight_vehicle_approval(exceed_weight_mix)

In [41]:
def update_loadability_and_vehicle_mixed(newclub, exceed_weight_mix):
    for i in newclub.index:
        for j in exceed_weight_mix.index:
            if newclub.loc[i, 'Cluster_No'] == exceed_weight_mix.loc[j, 'Cluster_No']:
                newclub.loc[i, 'Loadability_Loss'] = exceed_weight_mix.loc[j, 'Loadability_Loss']
                newclub.loc[i, 'Vehicle'] = exceed_weight_mix.loc[j, 'Vehicle']
                newclub.loc[i, 'Vehicle Type'] = exceed_weight_mix.loc[j, 'Vehicle Type']
                newclub.loc[i, 'Final_DD'] = exceed_weight_mix.loc[j, 'Expected DispatchD']
                newclub.loc[i, 'Max_Weight_Kgs'] = exceed_weight_mix.loc[j, 'Max_Weight_Kgs']
                newclub.loc[i, 'Min_Weight_Kgs'] = exceed_weight_mix.loc[j, 'Min_Weight_Kgs']
                newclub.loc[i, 'Approval Condition'] = exceed_weight_mix.loc[j, 'Approval Condition']
    
    return newclub
newclub = update_loadability_and_vehicle_mixed(newclub, exceed_weight_mix)

In [42]:
def process_review_conditions(df_filtered3, newclub):
    mix_thin_thick = df_filtered3.merge(newclub[['Indent No.', 'Cluster_No', 'Vehicle', 'Vehicle Type', 'Max_Weight_Kgs', 'Min_Weight_Kgs', 'Loadability_Loss', 'Approval Condition', 'Loading_order', 'Final_DD']], on='Indent No.', how='left')
    
    review_condition = mix_thin_thick.groupby('Cluster_No').filter(lambda x: (1 in x['Thin'].values) and (0 in x['Thin'].values))
    review_condition['Total_Gross_Weight_Thin_1'] = review_condition[review_condition['Thin'] == 1].groupby('Cluster_No')['Gross Weight'].transform('sum')
    
    condition_18MT = (
        (review_condition['Thin'] == 1) &
        (review_condition['Vehicle'].str.startswith('18 MT')) &
        (review_condition['Total_Gross_Weight_Thin_1'] > 7000)
    )

    condition_23MT = (
        (review_condition['Thin'] == 1) &
        (review_condition['Vehicle'].str.startswith('23 MT')) &
        (review_condition['Total_Gross_Weight_Thin_1'] > 9500)
    )

    condition_24MT = (
        (review_condition['Thin'] == 1) &
        (review_condition['Vehicle'].str.startswith('24 MT')) &
        (review_condition['Total_Gross_Weight_Thin_1'] > 8000)
    )

    condition_30MT = (
        (review_condition['Thin'] == 1) &
        (review_condition['Vehicle'].str.startswith('30 MT')) &
        (review_condition['Total_Gross_Weight_Thin_1'] > 12000)
    )

    combined_condition = condition_18MT | condition_23MT | condition_24MT | condition_30MT
    clusters_to_review = review_condition.loc[combined_condition, 'Cluster_No'].unique()
    
    review_condition = mix_thin_thick[mix_thin_thick['Cluster_No'].isin(clusters_to_review)].reset_index(drop=True)
    approval_conditions_16 = []

    for index, row in review_condition.iterrows():
        minweight_str = str(row['Min_Weight_Kgs'])  
        maxweight_str = str(row['Max_Weight_Kgs'])  

        minweight_list = [float(weight) for weight in minweight_str.split(',') if weight.strip()]
        maxweight_list = [float(weight) for weight in maxweight_str.split(',') if weight.strip()]

        conditions_for_vehicles = []

        last_min_weight = minweight_list[-1]
        last_max_weight = maxweight_list[-1]
        weight1 = last_max_weight - last_min_weight

        for idx, (minweight, maxweight) in enumerate(zip(minweight_list, maxweight_list)):
            if idx < len(minweight_list) - 1:
                conditions_for_vehicles.append('Review')
            else:
                if weight1 > row['Loadability_Loss']:
                    conditions_for_vehicles.append('Review')
                elif row['Loadability_Loss'] < 1000:
                    conditions_for_vehicles.append('Review')
                elif row['Loadability_Loss'] > 1000:
                    conditions_for_vehicles.append('Review')
                else:
                    conditions_for_vehicles.append('Review')

        approval_conditions_16.append(', '.join(conditions_for_vehicles))

    if len(approval_conditions_16) == len(review_condition):
        review_condition['Approval Condition'] = approval_conditions_16

    return review_condition, mix_thin_thick
review_condition,mix_thin_thick = process_review_conditions(df_filtered3, newclub)

In [43]:
def update_approval_condition(mix_thin_thick, review_condition):
    for i in mix_thin_thick.index:
        for j in review_condition.index:
            if mix_thin_thick.loc[i, 'Cluster_No'] == review_condition.loc[j, 'Cluster_No']:
                mix_thin_thick.loc[i, 'Approval Condition'] = review_condition.loc[j, 'Approval Condition']
    return mix_thin_thick
mix_thin_thick = update_approval_condition(mix_thin_thick, review_condition)
# mix_thin_thick

In [44]:
def calculate_final_dispatch_dates(df_filtered, Sum_thin_GW, df_filtered2, Sum_thick_GW, mix_thin_thick):
    thinpannel = df_filtered.merge(Sum_thin_GW[['Indent No.', 'Cluster_No', 'Vehicle', 'Vehicle Type', 'Max_Weight_Kgs', 'Min_Weight_Kgs', 'Loadability_Loss', 'Approval Condition', 'Loading_order', 'Final_DD']], on='Indent No.', how='left') 
    Thick = df_filtered2.merge(Sum_thick_GW[['Indent No.', 'Cluster_No', 'Vehicle', 'Vehicle Type', 'Max_Weight_Kgs', 'Min_Weight_Kgs', 'Loadability_Loss', 'Approval Condition', 'Loading_order', 'Final_DD']], on='Indent No.', how='left')
    ThinThick = pd.concat([thinpannel, Thick], ignore_index=True)
    ThinThick = pd.concat([ThinThick, mix_thin_thick], ignore_index=True)
    current_hour = dt.datetime.now().time().hour
    if current_hour >= 16:
        ThinThick['Final_DD'] = ThinThick['Final_DD'] + dt.timedelta(days=1) 

    gs03_finalDD = ThinThick[ThinThick['gs03_Agrade'].notna() & (ThinThick['gs03_available_stock'] > 0)]
    gs03_finalDD = gs03_finalDD.reset_index(drop=True)  
    current_hour = dt.datetime.now().time().hour
    if current_hour >= 5: 
        gs03_finalDD['Final_DD'] = gs03_finalDD['Expected DispatchD'] + dt.timedelta(days=2)   

    gs02_finalDD_4D = ThinThick[ThinThick['gs02_Agrade'].notna() & (ThinThick['Thickness'] <= 5.5) & (ThinThick['gs02_available_stock'] > 0)]
    gs02_finalDD_4D = gs02_finalDD_4D.reset_index(drop=True) 

    current_hour = dt.datetime.now().time().hour
    if current_hour >= 5:
        gs02_finalDD_4D['Final_DD'] = gs02_finalDD_4D['Expected DispatchD'] + dt.timedelta(days=3) 

    gs02_finalDD_5D = ThinThick[ThinThick['gs02_Agrade'].notna() & (ThinThick['Thickness'] > 5.5) & (ThinThick['Thickness'] <= 11) & (ThinThick['gs02_available_stock'] > 0)]
    gs02_finalDD_5D = gs02_finalDD_5D.reset_index(drop=True) 

    current_hour = dt.datetime.now().time().hour
    if current_hour >= 5:
        gs02_finalDD_5D['Final_DD'] = gs02_finalDD_5D['Expected DispatchD'] + dt.timedelta(days=4) 

    gs02_finalDD_6D = ThinThick[ThinThick['gs02_Agrade'].notna() & (ThinThick['Thickness'] > 11) & (ThinThick['gs02_available_stock'] > 0)]
    gs02_finalDD_6D = gs02_finalDD_6D.reset_index(drop=True)

    current_hour = dt.datetime.now().time().hour 
    if current_hour >= 5:
        gs02_finalDD_6D['Final_DD'] = gs02_finalDD_6D['Expected DispatchD'] + dt.timedelta(days=5) 

    gs02_finalDD = ThinThick[ThinThick['gs02_Agrade'].notna() & (ThinThick['gs02_available_stock'] > 0)] 
    gs02_finalDD = gs02_finalDD.reset_index(drop=True)

    return ThinThick, gs03_finalDD, gs02_finalDD_4D, gs02_finalDD_5D, gs02_finalDD_6D, gs02_finalDD
ThinThick, gs03_finalDD, gs02_finalDD_4D, gs02_finalDD_5D, gs02_finalDD_6D, gs02_finalDD = calculate_final_dispatch_dates(df_filtered, Sum_thin_GW, df_filtered2, Sum_thick_GW, mix_thin_thick)

In [45]:
def update_final_dispatch_dates(ThinThick, gs03_finalDD, gs02_finalDD_4D, gs02_finalDD_5D, gs02_finalDD_6D):
    for i in ThinThick.index:
        for j in gs03_finalDD.index: 
            if ThinThick.loc[i,'Cluster_No'] == gs03_finalDD.loc[j,'Cluster_No']:
                if ThinThick.loc[i,'Material'] == gs03_finalDD.loc[j,'Material']:
                    if ThinThick.loc[i,'Indent No.'] == gs03_finalDD.loc[j,'Indent No.']: 
                        ThinThick.loc[i,'Final_DD'] = gs03_finalDD.loc[j,'Final_DD']

    for i in ThinThick.index:
        for j in gs02_finalDD_4D.index: 
            if ThinThick.loc[i,'Cluster_No'] == gs02_finalDD_4D.loc[j,'Cluster_No']:
                if ThinThick.loc[i,'Material'] == gs02_finalDD_4D.loc[j,'Material']:
                    if ThinThick.loc[i,'Indent No.'] == gs02_finalDD_4D.loc[j,'Indent No.']: 
                        ThinThick.loc[i,'Final_DD'] = gs02_finalDD_4D.loc[j,'Final_DD']

    for i in ThinThick.index:
        for j in gs02_finalDD_5D.index: 
            if ThinThick.loc[i,'Cluster_No'] == gs02_finalDD_5D.loc[j,'Cluster_No']:
                if ThinThick.loc[i,'Material'] == gs02_finalDD_5D.loc[j,'Material']:
                    if ThinThick.loc[i,'Indent No.'] == gs02_finalDD_5D.loc[j,'Indent No.']: 
                        ThinThick.loc[i,'Final_DD'] = gs02_finalDD_5D.loc[j,'Final_DD']

    for i in ThinThick.index:
        for j in gs02_finalDD_6D.index: 
            if ThinThick.loc[i,'Cluster_No'] == gs02_finalDD_6D.loc[j,'Cluster_No']:
                if ThinThick.loc[i,'Material'] == gs02_finalDD_6D.loc[j,'Material']:
                    if ThinThick.loc[i,'Indent No.'] == gs02_finalDD_6D.loc[j,'Indent No.']: 
                        ThinThick.loc[i,'Final_DD'] = gs02_finalDD_6D.loc[j,'Final_DD']

    return ThinThick
ThinThick = update_final_dispatch_dates(ThinThick, gs03_finalDD, gs02_finalDD_4D, gs02_finalDD_5D, gs02_finalDD_6D)

In [46]:
def process_shortened_data(ThinThick, thick_full_size_10x4, thin_full_size_10x4, mixed_material_10x4):
     
    for i in range(len(ThinThick)):
        if isinstance(ThinThick.loc[i, 'Approval Condition'], str) and ThinThick.loc[i, 'Approval Condition'].startswith(('Reject', 'Review')):
            ThinThick.loc[i, 'Final_DD'] = 'NA' 
        else:
            pass

    columns_to_drop = ['Expected DispatchD']
    ThinThick.drop(columns=columns_to_drop, inplace=True) 
    Shorted = ThinThick.sort_values(by='Cluster_No')    
    Shorted['Total_Gross_weight'] = Shorted.groupby('Cluster_No')['Gross Weight'].transform('sum')
    clusters_with_10x4 = Shorted.loc[
        Shorted['Material Description'].str.contains(('10X4|10x4')) & Shorted['Cluster_No'].notna(), 
        'Cluster_No'
    ].unique()

    Shorted_10x4 = Shorted.loc[
        Shorted['Cluster_No'].isin(clusters_with_10x4) & Shorted['Cluster_No'].notna()
    ].reset_index(drop=True)

    Shorted_10x4_10MT = Shorted_10x4.loc[
        Shorted_10x4['Vehicle'] =='10 MT'
    ].reset_index(drop=True) 

    Shorted_10x4_10MT_rest = Shorted.loc[
        ~Shorted.index.isin(Shorted_10x4_10MT.index)
    ].reset_index(drop=True)
    Shorted_10x4_10MT_both = Shorted_10x4_10MT.groupby('Cluster_No').filter(lambda group: (1 in group['Thin'].values) and (0 in group['Thin'].values))
    Shorted_10x4_10MT_thin = Shorted_10x4_10MT[~Shorted_10x4_10MT.index.isin(Shorted_10x4_10MT_both.index)]
    Shorted_10x4_10MT_thick = Shorted_10x4_10MT_thin[Shorted_10x4_10MT_thin['Thin'] == 1]
    Shorted_10x4_10MT_thin = Shorted_10x4_10MT_thin[Shorted_10x4_10MT_thin['Thin'] == 0]

    columns_to_reset = ['Vehicle', 'Vehicle Type', 'Max_Weight_Kgs', 'Min_Weight_Kgs', 'Loadability_Loss', 'Approval Condition', 'Final_DD']
    Shorted_10x4_10MT.loc[:, columns_to_reset] = 0                     

    df_10x4 = Shorted_10x4_10MT[Shorted_10x4_10MT['Material Description'].str.contains('10X4|10x4')]

    df_10x4_grouped = df_10x4.groupby('Cluster_No')['Gross Weight'].sum().reset_index()
    df_10x4_grouped.rename(columns={'Gross Weight': '10x4_total_gross_weight'}, inplace=True)

    Shorted_10x4_10MT = Shorted_10x4_10MT.merge(df_10x4_grouped, on='Cluster_No', how='left')

    for i in range(len(Shorted_10x4_10MT)):
        cluster_no = Shorted_10x4_10MT.loc[i, 'Cluster_No']
        weight = Shorted_10x4_10MT.loc[i, 'Gross Weight']
        distance = Shorted_10x4_10MT.loc[i, 'Distance']

        if cluster_no > 0:
            if cluster_no in Shorted_10x4_10MT_thin['Cluster_No'].values:  
                df = thick_full_size_10x4
            elif cluster_no in Shorted_10x4_10MT_thick['Cluster_No'].values:
                df = thin_full_size_10x4
            elif cluster_no in Shorted_10x4_10MT_both['Cluster_No'].values:
                df = mixed_material_10x4

            for j in range(len(df) - 1, -1, -1):
                if weight <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Shorted_10x4_10MT.loc[i, 'Vehicle'] = df.loc[j, 'Vehicle Capacity in MT']
                    Shorted_10x4_10MT.loc[i, 'Min_Weight_Kgs'] = df.loc[j, 'Min_Weight_Kgs']
                    Shorted_10x4_10MT.loc[i, 'Max_Weight_Kgs'] = df.loc[j, 'Max_Weight_Kgs']
                    Shorted_10x4_10MT.loc[i, 'Vehicle Type'] = df.loc[j, 'Vehicle Type']
                    Shorted_10x4_10MT.loc[i, 'Loadability_Loss'] = df.loc[j, 'Max_Weight_Kgs'] - weight
                    Shorted_10x4_10MT.loc[i, 'review_limit'] = df.loc[j, 'review_limit']

    for i in range(len(Shorted_10x4_10MT)):
        cluster_no = Shorted_10x4_10MT.loc[i, 'Cluster_No']
        weight = Shorted_10x4_10MT.loc[i, 'Gross Weight']
        distance = Shorted_10x4_10MT.loc[i, 'Distance']

        if cluster_no <= 0:
            if cluster_no in Shorted_10x4_10MT_thin['Cluster_No'].values:
                df = thick_full_size_10x4
            elif cluster_no in Shorted_10x4_10MT_thick['Cluster_No'].values:
                df = thin_full_size_10x4
            elif cluster_no in Shorted_10x4_10MT_both['Cluster_No'].values:
                df = mixed_material_10x4

            for j in range(len(df) - 1, -1, -1):
                if weight <= df.loc[j, 'Max_Weight_Kgs'] and distance <= df.loc[j, 'Max Distance']:
                    Shorted_10x4_10MT.loc[i, 'Vehicle'] = df.loc[j, 'Vehicle Capacity in MT']
                    Shorted_10x4_10MT.loc[i, 'Min_Weight_Kgs'] = df.loc[j, 'Min_Weight_Kgs']
                    Shorted_10x4_10MT.loc[i, 'Max_Weight_Kgs'] = df.loc[j, 'Max_Weight_Kgs']
                    Shorted_10x4_10MT.loc[i, 'Vehicle Type'] = df.loc[j, 'Vehicle Type']
                    Shorted_10x4_10MT.loc[i, 'Loadability_Loss'] = df.loc[j, 'Max_Weight_Kgs'] - weight
                    Shorted_10x4_10MT.loc[i, 'review_limit'] = df.loc[j, 'review_limit']

    return Shorted, Shorted_10x4_10MT
Shorted, Shorted_10x4_10MT = process_shortened_data(ThinThick, thick_full_size_10x4, thin_full_size_10x4, mixed_material_10x4)

In [47]:
def predict_edd(Shorted, Shorted_10x4_10MT):
    approval_conditions10001 = []

    for i in range(len(Shorted_10x4_10MT)):
        weight1 = sum(Shorted_10x4_10MT[Shorted_10x4_10MT['Cluster_No'] == Shorted_10x4_10MT['Cluster_No'][i]]['Gross Weight'])
        minweight = Shorted_10x4_10MT['Min_Weight_Kgs'][i]
        maxweight = Shorted_10x4_10MT['Max_Weight_Kgs'][i]
        total_gross_weight = Shorted_10x4_10MT['Total_Gross_weight'][i] 
        review_limit = Shorted_10x4_10MT['review_limit'][i] 

        if minweight <= total_gross_weight <= maxweight:
            approval_conditions10001.append('Approved')
        elif total_gross_weight < review_limit:
            approval_conditions10001.append('Reject') 
        elif minweight > total_gross_weight >= review_limit:
            approval_conditions10001.append('Review') 
        else:
            approval_conditions10001.append('NA')  

    if len(approval_conditions10001) == len(Shorted_10x4_10MT):
        Shorted_10x4_10MT['Approval Condition'] = approval_conditions10001 
    Shorted_10x4_10MT = Shorted_10x4_10MT[Shorted_10x4_10MT['10x4_total_gross_weight'] > 5000].reset_index()

    for i in Shorted.index:    
        for j in Shorted_10x4_10MT.index:
            if Shorted.loc[i,'Cluster_No'] == Shorted_10x4_10MT.loc[j,'Cluster_No']:
                Shorted.loc[i,'Loadability_Loss'] = Shorted_10x4_10MT.loc[j,'Loadability_Loss']
                Shorted.loc[i,'Vehicle'] = Shorted_10x4_10MT.loc[j,'Vehicle']
                Shorted.loc[i,'Vehicle Type'] = Shorted_10x4_10MT.loc[j,'Vehicle Type']
                Shorted.loc[i,'Max_Weight_Kgs'] = Shorted_10x4_10MT.loc[j,'Max_Weight_Kgs'] 
                Shorted.loc[i,'Min_Weight_Kgs'] = Shorted_10x4_10MT.loc[j,'Min_Weight_Kgs']
                Shorted.loc[i,'Approval Condition'] = Shorted_10x4_10MT.loc[j,'Approval Condition'] 
    return Shorted,Shorted_10x4_10MT 
Shorted,Shorted_10x4_10MT = predict_edd(Shorted, Shorted_10x4_10MT)
Shorted.to_excel("EDD_Output19oct.xlsx")